<a href="https://colab.research.google.com/github/anirudh6370/Dog-VS-Cat-classifier/blob/main/Dog_vs_cat_vgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [22]:
!kaggle datasets download -d salader/dogs-vs-cats

dogs-vs-cats.zip: Skipping, found more recently modified local copy (use --force to force download)


In [23]:
import zipfile
zip_ref = zipfile.ZipFile(r'/content/dogs-vs-cats.zip')
zip_ref.extractall('/content/')
zip_ref.close()

In [24]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Flatten,Dense

In [25]:
conv_base = VGG16(
  weights = 'imagenet',
  include_top = False,
  input_shape = (150,150,3))

In [26]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [27]:
model = Sequential()
model.add(conv_base)
model.add(Flatten())
model.add(Dense(128,activation = 'relu'))
model.add(Dense(1,activation = 'sigmoid'))

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 8192)              0         
                                                                 
 dense_2 (Dense)             (None, 128)               1048704   
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 15763521 (60.13 MB)
Trainable params: 15763521 (60.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
conv_base.trainable = False

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 8192)              0         
                                                                 
 dense_2 (Dense)             (None, 128)               1048704   
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 15763521 (60.13 MB)
Trainable params: 1048833 (4.00 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [31]:
train_ds = keras.utils.image_dataset_from_directory(
        directory = '/content/train',
        labels = 'inferred',
        label_mode = 'int',
        batch_size = 32,
        image_size = (150,150)
)

Found 20000 files belonging to 2 classes.


In [32]:
valid_ds = keras.utils.image_dataset_from_directory(
        directory = '/content/test',
        labels = 'inferred',
        label_mode = 'int',
        batch_size = 32,
        image_size = (150,150)
)

Found 5000 files belonging to 2 classes.


In [33]:


#normalization
def Preprocess(image,label):
  image = tf.cast(image/255.,tf.float32)
  return image,label


In [34]:
train_ds = train_ds.map(Preprocess)
valid_ds = valid_ds.map(Preprocess)

In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_1 (Flatten)         (None, 8192)              0         
                                                                 
 dense_2 (Dense)             (None, 128)               1048704   
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 15763521 (60.13 MB)
Trainable params: 1048833 (4.00 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [36]:
model.compile(loss = 'binary_crossentropy',optimizer = 'adamax',metrics = ['accuracy'])

In [37]:
model.fit(train_ds,epochs = 10,validation_data = valid_ds)

Epoch 1/10
625/625 [==============================] - 67s 105ms/step - loss: 0.2565 - accuracy: 0.8905 - val_loss: 0.2150 - val_accuracy: 0.9068
Epoch 2/10
625/625 [==============================] - 65s 103ms/step - loss: 0.1951 - accuracy: 0.9177 - val_loss: 0.2025 - val_accuracy: 0.9128
Epoch 3/10
625/625 [==============================] - 55s 87ms/step - loss: 0.1709 - accuracy: 0.9293 - val_loss: 0.2029 - val_accuracy: 0.9154
Epoch 4/10
625/625 [==============================] - 65s 103ms/step - loss: 0.1485 - accuracy: 0.9395 - val_loss: 0.2073 - val_accuracy: 0.9148
Epoch 5/10
625/625 [==============================] - 64s 103ms/step - loss: 0.1304 - accuracy: 0.9485 - val_loss: 0.2064 - val_accuracy: 0.9150
Epoch 6/10
625/625 [==============================] - 55s 88ms/step - loss: 0.1139 - accuracy: 0.9567 - val_loss: 0.2135 - val_accuracy: 0.9160
Epoch 7/10
625/625 [==============================] - 54s 86ms/step - loss: 0.0983 - accuracy: 0.9618 - val_loss: 0.2391 - val_accur

In [38]:
# prompt: predict for test set

test_ds = keras.utils.image_dataset_from_directory(
        directory = '/content/test',
        labels = 'inferred',
        label_mode = 'int',
        batch_size = 32,
        image_size = (150,150)
)
test_ds = test_ds.map(Preprocess)
test_pred = model.predict(test_ds)
print(test_pred)


Found 5000 files belonging to 2 classes.
157/157 [==============================] - 11s 68ms/step
[[4.7345902e-06]
 [9.9569242e-03]
 [9.9974459e-01]
 ...
 [2.9100734e-01]
 [9.9542052e-01]
 [4.6745941e-01]]


In [39]:
# prompt: now according to the result show the accuracy

model.evaluate(test_ds)


157/157 [==============================] - 11s 68ms/step - loss: 0.2418 - accuracy: 0.9128


[0.24177956581115723, 0.9128000140190125]